In [1]:
# Import modules
import os                                    # for working with files
import time                                  # for time-related functions
import torch                                 # for the Pytorch module
from torch import nn                         # for creating neural networks 
from torch import optim                      # for optimization algorithms.
from torch.utils.data import DataLoader      # for dataloaders
from torchvision import transforms           # for transforming images into tensors
from torchvision import models               # for pre-trained models
from torchvision.datasets import ImageFolder # for working with classes and images

In [2]:
# Define the dataset path
data_path = "dataset/dataset"
train_path = data_path + "/train"
valid_path = data_path + "/valid"

In [3]:
# Define the data transforms
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "valid": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

In [4]:
# Load the dataset
train_data = ImageFolder(train_path, transform=data_transforms["train"])
valid_data = ImageFolder(valid_path, transform=data_transforms["valid"])

In [5]:
# Data shape
shape = train_data[0][0].shape
shape

torch.Size([3, 224, 224])

In [6]:
# Set the batch size
# Scale according to System Memory(CPU) or GPU Memory(GPU)
batch_size = 64 #16

In [7]:
# Set num_workers and pin_memory 
# Scale according to System Memory
n_ws = 4
p_m = True

In [8]:
# Define the dataset dataloaders
train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=n_ws, pin_memory=p_m)
valid_loader = DataLoader(valid_data, batch_size, shuffle=True, num_workers=n_ws, pin_memory=p_m)

In [9]:
# Select the device to use for processing
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
device

device(type='cpu')

In [10]:
# Set cuDNN to benchmark multiple convolution algorithms and select the fastest
# only applicable if a CUDA device is selected for processing
#if (device.type == "cuda"):
#    torch.backends.cudnn.benchmark = True
#    print("True")

In [11]:
# Define the EfficientNetV2-S model
model = models.mobilenet_v3_large(weights="DEFAULT")

In [12]:
# Number of features in the last layer
num_ftrs = model.classifier[-1].in_features
num_ftrs

1280

In [13]:
# Number of classes in train data
num_clss = len(train_data.classes)
print(train_data.classes)
num_clss

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

38

In [14]:
# Replace the last fully connected layer with a new one
model.classifier[-1] = nn.Linear(num_ftrs, num_clss)

In [15]:
# Move the model to the device
model = model.to(device)

In [16]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
# Initialize the metrics
current_epoch = 0
train_loss = 0.0
valid_loss = 0.0
valid_acc = 0.0
timer = 0.0

In [18]:
# # Load the model checkpoint
# # Uncomment the cell to resume training the model
# # Else the model training will be reset
# checkpoint = torch.load("models/model_resumable.pth")
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# current_epoch = checkpoint["current_epoch"]
# train_loss = checkpoint["train_loss"]
# valid_loss = checkpoint["valid_loss"]
# valid_acc = checkpoint["valid_acc"]
# timer = checkpoint["timer"]

# # Print the metrics of the final epoch of the last training session
# print(("Epoch {}: Training Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.2f}%, Time: {:.2f}s"
# ).format((current_epoch), train_loss, valid_loss, valid_acc, timer))

In [19]:
# Set the number of epochs for the current training session
num_epoch = 3

In [20]:
# Train the model
# Required epoch upto which to train
required_epoch = num_epoch + current_epoch

# Model training
for epoch in range(current_epoch, required_epoch):
    # Initialize the metrics
    train_loss = 0.0
    valid_loss = 0.0
    valid_acc = 0.0

    # Start time
    start_time = time.time()

    # Training step
    # Set the model to training mode
    model.train()
    for inputs, labels in train_loader:
        # Move the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad(set_to_none=True)

        # Backward pass
        loss.backward()

        # Update weigths every other iteration
        optimizer.step()

        # Compute the training loss
        train_loss += loss.item()
    
    # Validation step
    # Set the model to evaluation mode
    model.eval()
    # Disable gradients
    with torch.no_grad():
        for inputs, labels in valid_loader:
            # Move the data to the device
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Compute the validation loss
            valid_loss += loss.item()

            # Compute the accuracy
            _, preds = torch.max(outputs, 1)
            valid_acc += (preds == labels).sum().item()

    # End time
    end_time = time.time()

    # Execution time
    timer = end_time - start_time
        
    # Compute the average training loss and validation loss, and accuracy percentage
    train_loss /= len(train_data)
    valid_loss /= len(valid_data)
    valid_acc = (valid_acc / len(valid_data))*100

    # Print the metrics of the current epoch of the current training session
    print(("Epoch {}: Training Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.2f}%, Time: {:.2f}s"
    ).format((epoch + 1), train_loss, valid_loss, valid_acc, timer))

# Update current epoch
current_epoch = epoch + 1

Epoch 1: Training Loss: 0.0163, Validation Loss: 0.0023, Accuracy: 95.95%, Time: 5102.06s
Epoch 2: Training Loss: 0.0031, Validation Loss: 0.0013, Accuracy: 97.81%, Time: 5170.79s
Epoch 3: Training Loss: 0.0022, Validation Loss: 0.0009, Accuracy: 98.17%, Time: 5175.00s


In [21]:
# Save the model
torch.save(model.state_dict(), "models/model.pth")

In [22]:
# Save the model(complete)
torch.save(model, "models/model_complete.pth")

In [23]:
# Save the model(resumable)
torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "current_epoch": current_epoch,
            "train_loss": train_loss,
            "valid_loss": valid_loss,
            "valid_acc": valid_acc,
            "train_loss": train_loss,
            "timer": timer
            }, "models/model_resumable.pth")

In [24]:
# Export the model to TorchScript(script)
model_scripted = torch.jit.script(model)
model_scripted.save("models/model_scripted.pt")

In [25]:
# Export the model to TorchScript(trace)
example_input = torch.rand((1,) + tuple(shape))
example_input = example_input.to(device)
with torch.jit.optimized_execution(True):
    model_traced = torch.jit.trace(model, example_input)
model_traced.save("models/model_traced.pt")